In [43]:
from pynq import Overlay, MMIO
import time

# Program the FPGA from ARM processor
conv_design = Overlay("./CONV16.bit")
conv_design.download()
conv_design.ip_dict


{'axi_cdma_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xb024d690>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'axi_cdma_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S_AXI_LITE',
  'parameters': {'C_ADDR_WIDTH': '32',
   'C_AXI_LITE_IS_ASYNC': '0',
   'C_BASEADDR': '0x7E200000',
   'C_DLYTMR_RESOLUTION': '256',
   'C_ENABLE_KEYHOLE': 'false',
   'C_FAMILY': 'zynq',
   'C_HIGHADDR': '0x7E20FFFF',
   'C_INCLUDE_DRE': '0',
   'C_INCLUDE_SF': '0',
   'C_INCLUDE_SG': '0',
   'C_M_AXI_ADDR_WIDTH': '32',
   'C_M_AXI_DATA_WIDTH': '32',
   'C_M_AXI_MAX_BURST_LEN': '16',
   'C_M_AXI_SG_ADDR_WIDTH': '32',
   'C_M_AXI_SG_DATA_WIDTH': '32',
   'C_READ_ADDR_PIPE_DEPTH': '4',
   'C_S_AXI_LITE_ADDR_WIDTH': '6',
   'C_S_AXI_LITE_DATA_WIDTH': '32',
   'C_USE_DATAMOVER_LITE': '0',
   'C_WRITE_ADDR_PIPE_DEPTH': '4',
   'Component_Name': 'design_1_axi_cdma_0_0',
   'EDK_IPTYPE': 'PERIPHERAL'},
  'phys_addr': 2116026368,
  'registers': {'BTT': {'access': 'read-wri

In [44]:
# 設定各硬體模組的實體地址（Physical Address）
cdma_addr = conv_design.ip_dict['axi_cdma_0']['phys_addr']
gpio0_address = conv_design.ip_dict['axi_gpio_0']['phys_addr']
gpio1_address = conv_design.ip_dict['axi_gpio_1']['phys_addr']

# 定義控制信號的地址
start_conv_addr = gpio1_address + 0  # 啟動卷積的控制位址
fin_addr = gpio0_address + 0 # 結束訊號的位址

# print(hex(start_conv_addr))
# print(hex(fin_addr) )


In [45]:
# 定義資料傳輸區的記憶體映射地址

mem_addr = 0x10000000 # DDR 主記憶體起始位址
 
bram0_addr = 0xC0000000 # BRAM 0 的映射地址

bram1_addr = 0xC2000000 # BRAM 1 的映射地址



# 建立MMIO
in_bytes = 784*4 # 輸入資料大小（28x28 圖像）
in_mem = MMIO(mem_addr, in_bytes) 

out_bytes = 676*4 # 輸出資料大小（26x26 的卷積結果）
cdma = MMIO(cdma_addr, out_bytes)



start = MMIO(start_conv_addr, 4096) # 建立 GPIO 控制的 MMIO 物件（start 為啟動控制，done 為完成訊號）
done = MMIO(fin_addr ,4096)
done.write(0x4, 0x1)



In [46]:
# 從 input.hex 讀取輸入資料（通常為影像或其他模型輸入）
input_file = open('input.hex', 'r')
in_lines = input_file.readlines()
raddr = 0

# 將每行十六進位數字寫入DDR
for line in in_lines:
    hex_string = line[0:8]
    to_int = int(hex_string, 16)
    # Write to memory
    in_mem.write(raddr, to_int)
    output = hex(in_mem.read(raddr))
    raddr += 4

# 使用 CDMA 控制器從主記憶體 (DDR) 搬移資料到 BRAM 
cdma.write(0x00, 0x04)
cdma.write(0x18, mem_addr)
cdma.write(0x20, bram0_addr)
cdma.write(0x28, in_bytes)


In [47]:

# 將 GPIO 設為輸出模式：
# 向 GPIO 的 TRI（方向控制暫存器）寫入 0x00，代表為「輸出模式」
start.write(0x4, 0x0)  

# 向 GPIO 的 DATA 寄存器寫入 1，啟動硬體 IP
start.write(0x0, 0x1) 
time.sleep(1) 
# 將 GPIO 輸出清為 0
start.write(0x0, 0x0) 

# 等待 IP 運算完成
while True:
    value = done.read(0x0)
    done_bit = value & 0x1  
    if done_bit == 1:
        break
    time.sleep(0.001)  # 避免 CPU 忙碌等待

print(" IP 計算完成！")


 IP 計算完成！


In [48]:
# 使用 CDMA 控制器將資料從 BRAM1 搬回主記憶體（DDR）

# 對 CDMA reset，確保前一次傳輸不影響
cdma.write(0x00, 0x04)

# 設定來源位址為 BRAM1
cdma.write(0x18, bram1_addr)

#設定目標位址為 mem_addr
cdma.write(0x20, mem_addr)

# 設定要搬移的資料長度(卷積輸出的大小)
cdma.write(0x28, out_bytes)


In [49]:
golden_file = open('golden16.hex', 'r')
g_lines = golden_file.readlines()
err = 0
addr = 0

# 開啟一個新的檔案來寫回讀出的值

for line in g_lines: 
    expect = int(line.strip(), 16)
    output = in_mem.read(addr)

    # 寫入 readback.hex，每行為 8 位 hex 格式（補零對齊）
#     readback_file.write(f"{output & 0xFFFFFFFF:08X}\n")

    if output != expect:
        print(f"[Error] Output is {output:08X} expected {expect:08X} at addr {addr}")
        err += 1

    addr += 4

readback_file.close()

if err == 0:
    print("Congratulations! ALL pass 🎉")
else:
    print(f"Error! Data mismatch! Totally {err} errors.")



Congratulations! ALL pass 🎉
